# 第10章: ベクトル空間法 (II)

第10章では，前章に引き続き単語ベクトルの学習に取り組む．

In [1]:
from gensim.models import Word2Vec

# 90. word2vecによる学習
81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [2]:
# https://rare-technologies.com/word2vec-tutorial/
with open('81.txt', 'r') as f:
    corpus = [f.read().split()]
model = Word2Vec(corpus, min_count=1)    

In [3]:
# 86 ベクトル表示
model['United_States']

array([ -4.35321126e-03,   7.24946403e-06,  -1.00240693e-03,
         1.71894359e-03,   2.60639982e-03,   4.54979250e-03,
         2.76367622e-03,   1.16067892e-03,  -4.56093159e-03,
         2.28043390e-03,  -2.84206634e-03,   4.03411046e-04,
         2.00514449e-03,   3.51159112e-03,   4.71542217e-03,
        -1.17139041e-03,   5.38714649e-03,  -5.43413043e-04,
        -6.97838608e-04,   4.23885277e-03,  -2.26624170e-03,
        -2.62963469e-03,  -2.40363809e-03,  -1.88103202e-03,
        -4.21376433e-03,  -4.56124824e-03,  -3.34482896e-03,
        -4.16845037e-03,  -4.37373761e-03,   9.01171763e-04,
         4.30961140e-03,  -6.49189693e-04,  -1.51208998e-03,
        -1.78049924e-03,   1.85211143e-03,   3.75750707e-03,
        -5.62814949e-03,   1.86741853e-03,  -4.62834537e-03,
        -4.38138936e-03,  -7.36817543e-04,  -3.35923303e-03,
         3.46384384e-03,  -3.02884099e-03,   2.75690662e-04,
        -3.31420754e-03,   1.73456431e-03,  -2.17259116e-03,
        -1.79928832e-03,

In [4]:
# 87 コサイン類似度 
model.similarity("United_States", "U.S")

-0.067391358831258255

In [5]:
# 88 類似度 top10
model.most_similar(positive=['England'], topn=10)

[('Play/Create', 0.4888457953929901),
 ('Crossings"', 0.4459366798400879),
 ('conservatism"characterizing', 0.43599146604537964),
 ('Mathis"', 0.4313720464706421),
 ('springline', 0.41823285818099976),
 ('lila)', 0.4162786602973938),
 ('tape-manipulated', 0.41262805461883545),
 ('Piasecki', 0.4124037027359009),
 ('1909–1913)', 0.4096204340457916),
 ('Cen', 0.3987789452075958)]

In [6]:
# 89 アナロジー top10
model.most_similar(positive=['Spain', 'Athens'], negative=['Madrid'], topn=10)

[('TASC-01-SCF', 0.44708970189094543),
 ('dockworkers', 0.43916040658950806),
 ('Routemasters', 0.43558230996131897),
 ('title."</noinclude>', 0.4306522607803345),
 ('Standartenführer', 0.418748140335083),
 ('Trillions', 0.4170217216014862),
 ('Up")', 0.4168779253959656),
 ('"Bellum', 0.41194450855255127),
 ('Mutolo’s', 0.4111941456794739),
 ('factors', 0.4057306945323944)]

# 91. アナロジーデータの準備
単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [7]:
with open('questions-words.txt', 'r') as f:
    sections = f.read().split(':')
    for section in sections:
        if 'family' in section.split('\n')[0]:
            family = "\n".join(section.split('\n')[1:])
with open('family.txt', 'w') as f:
    f.write(family)

# 92. アナロジーデータへの適用
91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [8]:
with open('family.txt', 'r') as f:
    family = [x[:-1].split() for x in f.readlines()]
len(family)

506

In [9]:
def check_contain(words):
    for w in words:
        if not w in model:
            return False 
    return True

In [17]:
with open('92.txt', 'w') as f:
    for line in family:
        c1, c2, c3, c4 = line
        nomatch = model.doesnt_match(line)
        if check_contain(line):
            w, p = model.most_similar(positive=[c2, c3], negative=[c1], topn=1)[0]
            line = str(w) + "\t" + str(p) + "\n"
        else:
            line = "NoMatch\n"
        f.write(line)

# 93. アナロジータスクの正解率の計算
92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

In [29]:
count93 = 0
with open('92.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        w = line[:-1].split()[0]
        ans_w = family[i][3]
        if w != "NoMatch" and w == ans_w:
            count93 += 1
            print("match!!!!!")
count93 / len(family)

0.0

# 94. WordSimilarity-353での類似度計算
The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

# 95. WordSimilarity-353での評価
94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

# 96. 国名に関するベクトルの抽出
word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

# 97. k-meansクラスタリング
96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5k=5として実行せよ．

# 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

# 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．